In [ ]:
import pandas as pd
import pandas_profiling
import plotly.graph_objects as go

# Read data

In [ ]:
df = pd.read_csv('../data/data.csv', sep=',')
df.head()

In [ ]:
df['DEFECTIVE'].value_counts()

# Date formating

In [ ]:
df['DATE_ORDER'] = pd.to_datetime(df['DATE_ORDER'], format="%Y-%m-%d")
df['CONTACT_DATE'] = pd.to_datetime(df['CONTACT_DATE'], format="%Y-%m-%d")
df['PRODUCT_RELEASE_DATE'] = pd.to_datetime(df['PRODUCT_RELEASE_DATE'], format="%Y-%m-%d")

# EDA

Here we will have a look at a sample of data using `pandas_profiling`.  
It will allow us to get a quick overall view of each columns.  
This EDA is also available [here](https://tlentali.github.io/car_market/)

In [ ]:
df_sample = df.sample(frac=0.25, replace=False, random_state=1)
pandas_profiling.ProfileReport(df_sample)

# Filter `DEFECTIVE` on 30 days

In [ ]:
df['NB_DAYS_PROBLEM_START'] = (df['CONTACT_DATE'] - df['DATE_ORDER']).dt.days
df.head()

In [ ]:
fig = go.Figure(data=[go.Histogram(x=df['NB_DAYS_PROBLEM_START'])])
fig.show()

In [ ]:
df = df[(df['NB_DAYS_PROBLEM_START']<=30) | (df['NB_DAYS_PROBLEM_START'].isna())]
df.head()

In [ ]:
df['DEFECTIVE'].value_counts()

# `PRICE` cleanup

## Cut on `PRICE`

We will have a look on the `PRICE` distribution using different filter to get a better understanding of the data.

In [ ]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=df[df['DEFECTIVE'] == True]['PRICE']))
fig.add_trace(go.Histogram(x=df[df['DEFECTIVE'] == False]['PRICE']))

# The two histograms are drawn on top of another
fig.update_layout(barmode='stack')
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=df[df['PRODUCT_CATEGORY'] == 'motorcycle']['PRICE']))
fig.add_trace(go.Histogram(x=df[df['PRODUCT_CATEGORY'] == 'Car']['PRICE']))

# The two histograms are drawn on top of another
fig.update_layout(barmode='stack')
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=df[(df['DEFECTIVE'] == True) & (df['PRODUCT_CATEGORY'] == 'motorcycle')]['PRICE']))
fig.add_trace(go.Histogram(x=df[(df['DEFECTIVE'] == False) & (df['PRODUCT_CATEGORY'] == 'motorcycle')]['PRICE']))

# The two histograms are drawn on top of another
fig.update_layout(barmode='stack')
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=df[(df['DEFECTIVE'] == True) & (df['MODEL'] == 'HERITAGE Young')]['PRICE_NEW']))
fig.add_trace(go.Histogram(x=df[(df['DEFECTIVE'] == False) & (df['MODEL'] == 'HERITAGE Young')]['PRICE_NEW']))

# The two histograms are drawn on top of another
fig.update_layout(barmode='stack')
fig.show()

In [ ]:
fig = go.Figure(data=[go.Histogram(x=df['PRICE'])])
fig.show()

In [ ]:
df[df['PRICE']<50_000].sort_values(by='PRICE', ascending=False).head()

In [ ]:
len(df[df['PRICE']>60_000])

In [ ]:
df[df['PRICE']>60_000]['DEFECTIVE'].value_counts()

Here we can setup the fact that above 60_000 euro, the very thin tail in this distribution of price end.
Their is a lot of noise at around 406_000 euro to 450_000 euro that we can ride of.
By cutting from this price, we are losing 1536 rows, including 105 true defective items.

In [ ]:
df = df[df['PRICE']<=60_000]

In [ ]:
fig = go.Figure(data=[go.Histogram(x=df['PRICE'])])
fig.show()

## Cut on `PRICE_FROM_NEW_PERC`

Same exercice but on the `PRICE_FROM_NEW_PERC` column.

In [ ]:
df = df.copy()
df['PRICE_FROM_NEW_PERC'] = (df['PRICE'] - df['PRICE_NEW']) * 100 /  df['PRICE_NEW']
df.head()

In [ ]:
fig = go.Figure(data=[go.Histogram(x=df['PRICE_FROM_NEW_PERC'])])
fig.show()

In [ ]:
len(df[df['PRICE_FROM_NEW_PERC']>100])

In [ ]:
df[df['PRICE_FROM_NEW_PERC']>100]['DEFECTIVE'].value_counts()

Based on the distribution above, we are going to cut item that has been sold twice their origin price, losing 427 rows, including 49 true defective items.

In [ ]:
df = df[df['PRICE_FROM_NEW_PERC']<=100]

In [ ]:
fig = go.Figure(data=[go.Histogram(x=df['PRICE_FROM_NEW_PERC'])])
fig.show()

## New defective count

Check the new dataset obtained after applied filters on dates and prices.

In [ ]:
df['DEFECTIVE'].value_counts()

In [ ]:
df_sample = df.sample(frac=0.25, replace=False, random_state=1)
pandas_profiling.ProfileReport(df_sample)